# <font color='darkblue'> Part 2 : Problem Formulation / Patient Similarity Network Generation </font>

In this section we will formulate a task to predict if an individual in the Generation Scotland dataset is a smoker or not. We will formulate the task using the Patient Similarity Networks generated earlier. We will also highlight that not every network is created equal. 

Networks are flexible and we can leverage/incorporate additional informative information. We have used the [EWAS Catalog](https://www.ewascatalog.org/) to identify CpG sites affected by smoking. We will compare the effectiveness of GNN's when incorporating external informative versus doing a direct correlation network. 

## <font color='darkblue'>Import Functions and Packages </font>

In [ ]:
import pickle
import pandas as pd
import numpy as np
import networkx as nx
from palettable import wesanderson
import sys
sys.path.insert(0 , '/tutorial/')
from functions import *

## <font color='darkblue'>Import and Process Generation Scotland Data</font>

First we will load the data from Generation Scotland and create labels for individuals who have smoked before using the 'pack_years' feature

In [ ]:
# Load the Generation Scotland data using pickle
with open('/data/raw/ISMB_GS_Data.pkl' , 'rb') as file : 
    loaded_data = pd.read_pickle(file)

In [ ]:
'''
#################################################
               YOUR CODE HERE
#################################################

Extract 'DNAm_w1' (Generation Scotland Wave 1) from loaded_data pkl file, dropping any na's from the dataset

Extract 'Phenotypes' from loaded_data pkl file, set the index as `Sample_SentrixID` and subset to wave 1
'''

w1 = loaded_data[''].T.dropna() # we are only working with Wave 1 patients for this task
phenotypes = loaded_data[].set_index().loc[] # get the phenotypes for the wave 1 patients

In [ ]:
# Create a new column in the phenotypes dataframe that categorizes patients as either smokers or non-smokers
def smoking_cat(row) : 
    if row['pack_years'] == 0 :
        return 'Non-Smoker'
    else :
        return 'Smoker'

phenotypes['Smoking'] = phenotypes.apply(smoking_cat , axis = 1) # apply the function to the dataframe

In [ ]:
# For efficiency of graph training and learning we will downsample the dataset size to 1000 patients
never_smoked = phenotypes[phenotypes['Smoking'] == 'Non-Smoker'].sample(500) # sample 500 non-smokers
smoked = phenotypes[phenotypes['Smoking'] == 'Smoker'].sample(500) # sample 500 smokers

phenotypes = pd.concat([never_smoked , smoked]) # combine the two dataframes
w1 = w1.loc[phenotypes.index] # filter the methylation data to only include the patients in the downsampled phenotypes dataframe

In [ ]:
'''
#################################################
               YOUR CODE HERE
#################################################

Plot the distribution of smokers and non-smokers
'''

plt.title('Distribution of Smokers and Non-Smokers')

## <font color='darkblue'> EWAS Network Generation </font>

### <font color='darkblue'>Get a list of Smoking related CpG sites from the [EWAS Catalog](https://www.ewascatalog.org/)</font>

In this case we are looking at the smoking phenotype in a cohort from Generation Scotland. 

We have previously downloaded as tsv file for Smoking related CpG sites, but you can generate your own using the above link also.

Here we will extract the CpG's and related genes which we will use to build our network.

In [ ]:
df = pd.read_csv('/data/raw/smoking.tsv' , delimiter='\t') #Tab seperated document from EWAS
df.head()

### <font color='darkblue'>Patient Similarity Network Generation</font>

In [ ]:
'''
#################################################
                YOUR CODE HERE
#################################################

We want to include the most informative CpG sites, so we will only include CpG sites that have been annotated multiple times with smoking

1. Count the number of times a CpG site is present in the smoking.tcv dataset
2. If a CpG is present in the dataset more than 10 times add it to the list common_annotated_cpgs
3. Filter the methylation data to only include the CpG sites in common_annotated_cpgs
4. Call the new dataframe w1_filt

'''

In [ ]:
'''
#################################################
                YOUR CODE HERE
#################################################

We want to create our patient similarity matrix using the filtered methylation data
We will use biweight midcorrelation to calculate the similarity between patients 
The function for this is given as abs_bicorr

'''

In [ ]:
'''
#################################################
                YOUR CODE HERE
#################################################
We want to plot our Patient Similarity Network using the function plot_knn_network() from functions.py

Specify the node colours by assigning a colour to each phenotype using the .astype(`category`).cat.codes panda functionality

'''
node_colour =  # set the colours for the nodes

print(f"{phenotypes['Smoking'].astype('category').cat.categories[0]} : 0 \n{phenotypes['Smoking'].astype('category').cat.categories[1]} : 1") # print the mapping of the colours to the categories

G = plot_knn_network() # plot the network
plt.title('EWAS Smoking Patient Similarity Network')
legend_handles = gen_graph_legend(node_colour , G , 'label') # generate the legend
plt.legend(handles = legend_handles)
plt.show()

## <font color='darkblue'>Save Network and Data for GNN Learning </font>

In [ ]:
# Put the data in a single dictionary
data = {'Feat' : w1 , 'phenotypes' : phenotypes , 'PSN_EWAS' : G}

# File path to save the pickle file
file_path = '/data/intermediate/GCN_Data.pkl'

# Save data to a pickle file
with open(file_path, 'wb') as f:
    pickle.dump(data, f)